This notebook demonstrates how to process Sentinel-2 satellite imagery to calculate the Normalized Burn Ratio (NBR) and the delta Normalized Burn Ratio (dNBR) using the Google Earth Engine (GEE) Python API. 

- The **Normalized Burn Ratio (NBR)** is a remote sensing index used to identify areas affected by fire. It is computed using the near-infrared (NIR) and shortwave-infrared (SWIR) bands, which are sensitive to vegetation and burned areas.

- The **delta Normalized Burn Ratio (dNBR)** is the difference between pre-fire and post-fire NBR values, providing an effective metric for assessing burn severity.


## Workflow
1. Define the region of interest using a GeoJSON polygon.
2. Filter Sentinel-2 imagery by date and cloud cover.
3. Compute the NBR and dNBR indices.
4. Visualize the results with `geemap`

## Prerequisites

To run this notebook, you will need to have a Google Earth Engine account ([Sign up here](https://earthengine.google.com/)).

You will also need to ensure you have the necessary tools and libraries installed in a clean virtual environment. Follow the steps below to create and set up a virtual environment:  

### Install the dependencies

#### Using `venv` (Python Standard Library) 
1. **Create a virtual environment**:  
Ensure the desired Python version (e.g., Python 3.12) is installed on your system. Then run:

```bash  
   python3 -m venv nbr-env 
```
This creates a directory named nbr-env that contains the virtual environment.

1. **Activate the virtual environment**

- On macOS/Linux:
```bash

source nbr-env/bin/activate
```
- On Windows (Command Prompt):
```bash
nbr-env\Scripts\activate
```

3. **Install dependencies from requirements.txt**
```bash
pip install -r requirements.txt
```

####  Using conda (Anaconda/Miniconda)
1. Create a new Conda environment:

```bash
conda create --name nbr-env python=3.12
```
2. Activate the environment:

```bash
conda activate nbr-env
```

3. Use the following command to install dependencies:

```bash
pip install -r requirements.txt
```


Now you are all set up, let's start ! 

## Imports 

In [1]:
import ee
import json
import geemap

In [ ]:
# Authenticate
ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

## Define the Region of Interest

In [ ]:
# Load GeoJSON and and convert to Earth Engine geometry
def geojson_to_ee(file_path):
    with open(file_path, "r") as f:
        geojson_data = json.load(f)
    roi = ee.Geometry.Polygon(geojson_data["features"][0]["geometry"]["coordinates"])
    return roi


# Change the Geojson to your own need:
geojson_path = "./Los_Angeles.geojson"
roi = geojson_to_ee(geojson_path)
print("ROI set successfully:", roi.getInfo())

## Data processing

As of now the NBR and dNBR is computed using only Sentinel-2 imagery. Implementation with Landsat 8 will follow. 

In [ ]:
# Sentinel-2 Surface Reflectance and Cloud Probability datasets
S2_SR_COLLECTION = "COPERNICUS/S2_SR_HARMONIZED"
S2_CLOUD_PROB_COLLECTION = "COPERNICUS/S2_CLOUD_PROBABILITY"

def mask_s2_clouds(image, max_cloud_probability=60):
    """Masks clouds in a Sentinel-2 image using the COPERNICUS/S2_CLOUD_PROBABILITY dataset.

    Args:
        image (ee.Image): A Sentinel-2 image.
        max_cloud_probability (int, optional): Maximum cloud probability threshold. Defaults to 60.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    # Get cloud probability image matching the Sentinel-2 image
    cloud_prob = (
        ee.ImageCollection(S2_CLOUD_PROB_COLLECTION)
        .filterBounds(image.geometry())
        .filterDate(image.date(), image.date().advance(1, "day"))  # Match date
        .first()
    )
    
    # Ensure cloud probability image exists
    cloud_prob = ee.Image(cloud_prob)
    
    # If no cloud probability image is found, return the original image
    return ee.Algorithms.If(
        cloud_prob, 
        image.updateMask(cloud_prob.select("probability").lt(max_cloud_probability)), 
        image
    )

def mask_water_SCL(image):
    """Masks water in a Sentinel-2 image using  the SCL band.

    Args:
        image (ee.Image): Sentinel-2 image.

    Returns:
        ee.Image: Water-masked image.
    """
    scl = image.select("SCL")  # Scene Classification Layer
    water_mask = scl.neq(6)  # Water is class 6
    return image.updateMask(water_mask)

# Define a function to calculate NBR
def calculate_nbr(image):
    """Computes the Normalized Burn Ratio (NBR) for a Sentinel-2 image."""
    nbr = image.normalizedDifference(["B8", "B12"]).rename("NBR")
    return image.addBands(nbr)


# Retrieve Sentinel-2 images for a specific time range and region
def get_sentinel2_nbr(roi, start_date, end_date):
    sentinel2 = (
        ee.ImageCollection(S2_SR_COLLECTION)
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .map(mask_s2_clouds)  # Apply cloud masking
        .map(mask_water_SCL)  # Apply water masking
        .map(calculate_nbr)  # Compute NBR
    )

    # Check if the collection has valid images
    count = sentinel2.size().getInfo()
    if count == 0:
        raise ValueError(
            f"No images found for the date range {start_date} to {end_date}"
        )

    return sentinel2.mean()

In [ ]:
# Define the pre-event and post-event dates
pre_fire_start_date = "2024-12-01"
pre_fire_end_date = "2025-01-01"
post_fire_start_date = "2025-01-10"
post_fire_end_date = "2025-01-20"

pre_event_image = get_sentinel2_nbr(roi, pre_fire_start_date, pre_fire_end_date)
print(pre_event_image.getInfo())

post_event_image = get_sentinel2_nbr(roi, post_fire_start_date, post_fire_end_date)
print(post_event_image.getInfo())

In [ ]:
# Calculate dNBR for the given dates and ROI

# Compute dNBR
dnbr = (
    pre_event_image.select("NBR")
    .subtract(post_event_image.select("NBR"))
    .multiply(1000)
    .rename("dNBR")
)

In [ ]:
# Print the dNBR image details
print(dnbr.getInfo())

## Mapping the results

In [ ]:
Map = geemap.Map()
Map.centerObject(roi, 10)

boundary_style = {
    "color": "ffffff",  # White color
    "strokeWidth": 5,  # Stroke width of 5
}
Map.addLayer(roi, boundary_style, "Study Area")

# Define visualization parameters for NBR (display range, color palette)
nbr_vis = {"min": -1, "max": 1, "palette": ["blue", "white", "green"]}

# Add pre-event and post-event NBR images to the map
Map.addLayer(pre_event_image.select("NBR"), nbr_vis, "Pre-Fire NBR")
Map.addLayer(post_event_image.select("NBR"), nbr_vis, "Post-Fire NBR")

sld_intervals = """
<RasterSymbolizer>
  <ColorMap type="intervals" extended="false">
    <ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>
    <ColorMapEntry color="#7a8737" quantity="-250" label="-250"/>
    <ColorMapEntry color="#acbe4d" quantity="-100" label="-100"/>
    <ColorMapEntry color="#0ae042" quantity="100" label="100"/>
    <ColorMapEntry color="#fff70b" quantity="270" label="270"/>
    <ColorMapEntry color="#ffaf38" quantity="440" label="440"/>
    <ColorMapEntry color="#ff641b" quantity="660" label="660"/>
    <ColorMapEntry color="#a41fd6" quantity="2000" label="2000"/>
  </ColorMap>
</RasterSymbolizer>
"""

# Apply the SLD style for visualization
Map.addLayer(dnbr.sldStyle(sld_intervals), {}, "dNBR Classified (SLD)")

# Define the correct classification thresholds
thresholds = ee.Image([-500, -250, -100, 100, 270, 440, 660, 2000])

# Apply classification using thresholding
classified = dnbr.lt(thresholds).reduce("sum").toInt()


legend_dict = {
    "Enhanced Regrowth, High": "#7a8737",
    "Enhanced Regrowth, Low": "#acbe4d",
    "Unburned": "#0ae042",
    "Low Severity": "#fff70b",
    "Moderate-low Severity": "#ffaf38",
    "Moderate-high Severity": "#ff641b",
    "High Severity": "#a41fd6",
    "NA": "#ffffff",
}

# Add the legend to the map
Map.add_legend(
    title="dNBR Classes",
    keys=list(legend_dict.keys()),
    colors=list(legend_dict.values()),
)
# Show the map
Map

In [ ]:
# Define the export parameters
export_params_drive_greyscale = {
    "image": dnbr,
    "description": dnbr.id().getInfo(),
    "fileNamePrefix": "dNBR_Greyscale",
    "scale": 10,
    "region": roi,
    "maxPixels": 1e13,
}

export_params_drive_classified = {
    "image": classified,
    "description": dnbr.id().getInfo(),
    "fileNamePrefix": "dNBR_Classified",
    "scale": 10,
    "region": roi,
    "maxPixels": 1e13,
}

# Start the export tasks
task_drive_greyscale = ee.batch.Export.image.toDrive(**export_params_drive_greyscale)
task_drive_classified = ee.batch.Export.image.toDrive(**export_params_drive_classified)


# Start the tasks
task_drive_greyscale.start()
task_drive_classified.start()